In [29]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import LabelEncoder

In [30]:
train_goods = pd.concat([
    pd.read_csv('./训练集/traindata_goodsid/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./训练集/traindata_goodsid/part-00002', header=None, names=['goods_id', 'cat_id', 'brandsn'])
], axis=0)

train_user = pd.concat([
    pd.read_csv(x, header=None, names=['user_id', 'goods_id', 'is_clk', 'is_like', 'is_addcart', 'is_order', 'expose_start_time', 'dt'], nrows=500000)
    for x in glob.glob('./训练集/traindata_user/part*')
], axis=0)

In [31]:
testa_goods = pd.concat([
    pd.read_csv('./测试集a/predict_goods_id/part-00000', header=None, names=['goods_id', 'cat_id', 'brandsn']),
    pd.read_csv('./测试集a/predict_goods_id/part-00001', header=None, names=['goods_id', 'cat_id', 'brandsn']),
], axis=0)

testa_user = pd.read_excel('./测试集a/a榜需要预测的uid_5000.xlsx')

In [32]:
#计算测试集用户数据中的用户ID和商品ID在训练集用户数据中出现的比例
np.mean(testa_user['user_id'].isin(train_user['user_id'])), np.mean(testa_goods['goods_id'].isin(train_goods['goods_id']))

(1.0, 0.9999751455447659)

In [33]:
#去重
train_user['user_id'].nunique(), train_user['goods_id'].nunique()

(51602, 1531577)

In [34]:
#计算训练集用户数据的描述统计信息，并将结果保留两位小数
train_user.describe().round(2)

,is_clk,is_like,is_addcart,is_order,dt
count,7791816.00,7791816.00,7791816.00,7791816.00,7791816.00
mean,1.06,0.02,0.06,0.01,20230222.85
std,0.28,0.13,0.27,0.09,26.21
min,1.00,0.00,0.00,0.00,20230203.00
25%,1.00,0.00,0.00,0.00,20230209.00
50%,1.00,0.00,0.00,0.00,20230216.00
75%,1.00,0.00,0.00,0.00,20230224.00
max,121.00,22.00,28.00,15.00,20230303.00


In [35]:
#将训练集中的用户数据和商品数据根据商品ID进行合并
train_data = pd.merge(train_user.iloc[:], train_goods.iloc[:], on='goods_id')

In [36]:
#获取合并后的数据集中商品类别和品牌的数量
train_data['cat_id'].nunique(), train_data['brandsn'].nunique()
print(train_data)

                                  user_id                          goods_id  \
0        8da2ec07d8bf9bfe1e849cb7e7f25e5c  f6e4f43d18157cbdcdc653c6e35f01fb   
1        62a800cd3e6884078ac86af31926ca6e  f6e4f43d18157cbdcdc653c6e35f01fb   
2        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
3        dfea38a0a8c81429316857cbcede23c6  f6e4f43d18157cbdcdc653c6e35f01fb   
4        328395043a7cf38a36590a8df12b3b03  f6e4f43d18157cbdcdc653c6e35f01fb   
...                                   ...                               ...   
7791759  9b9c0493e6ee675c32b6c2add2b57cd9  49cb5e43cc1dc19cb5f5241cf06816a5   
7791760  2591ff4a1af75a2dc9e020cdafc5ab05  7fb4a90d3aa54ce13355c8ad9043e2e1   
7791761  18917649fd13d3a5fc37443c248d572e  a7e398eff0f997168bf341235ea4817d   
7791762  a4b0bc37d9cceed3bedc794e8d632d38  d6f2d674b397f89a55ecf34f127d0ad3   
7791763  333cb2e2e4c37293ca79f985a66d72f4  2acf9dbcd66278505f54da19d636825c   

         is_clk  is_like  is_addcart  is_order    e

In [37]:
#筛选出合并后的训练数据中满足以下条件的样本：未下单（is_order为0）且加购物车行为不为0（is_addcart不等于0）
train_agg_feat = train_data.loc[
    (train_data['is_order'] == 0) & (train_data['is_addcart'] != 0)
]
print(train_agg_feat)

                                  user_id                          goods_id  \
19       af640ab2869a652c05b0957d9a811704  81de1226af20e196944862b78d27b0b5   
76       11925e60aab947c3b00eee2190fa2ede  ff76680bd6e705fd37df578957faacba   
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
192      0189f417324cc706b727ae1944967c54  3eba0ec94cedb216bca77a07c6525d2d   
202      91aaaea6427f68f1338c6d3273304af2  a92320167ca0b39bea822c690e08c9d7   
...                                   ...                               ...   
7791652  a9ffeecb593fc6d7500ae1f565fda0f0  e1d97c03deb45d7cada039a08de92291   
7791670  353d9d983c3cd90068c59825622b5204  b3d3b112d865b3e99580b2ad49aca38d   
7791734  d2e569e2d1d8f7ad8d86d03ce8b534a0  5c366d4c5045ac3ff303191ddebaf95b   
7791744  fb2d2559a9e77372ffef7c9edebf87c5  c1b254a486172c86297e7a0af07981db   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [38]:
#筛选出train_agg_feat数据集中用户ID存在于测试集用户ID中的样本
train_agg_feat = train_agg_feat[train_agg_feat['user_id'].isin(testa_user['user_id'])]
print(train_agg_feat)

                                  user_id                          goods_id  \
191      c626a0405a9d85abdf05ccfd3d00e863  2d7e28b4f79bcd9de2b3df0bc91310a3   
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
...                                   ...                               ...   
7791220  9366b08f49ba3960eb6c58d41609cbd4  d805d53902739d835e6acac868f3ddbe   
7791271  884cfa576e9bd5e7c79307fdebd12ee4  73edfb067d6b0870214b7e51bfc630f5   
7791327  820b7afbd1f4e81a0fcbeee6d3123525  8805c368d93a74e0bdabe7832e8a9dee   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [39]:
#将train_agg_feat数据集中用户ID存在于测试集用户ID中的样本筛选出来
train_agg_feat = train_agg_feat[train_agg_feat['goods_id'].isin(testa_goods['goods_id'])]
print(train_agg_feat)

                                  user_id                          goods_id  \
705      060d0248975dfbe7dee433f32da5e266  ad182c11850f012948b84867362802be   
760      d01cfb26613e7ebfd53e8f50cfc2156a  5a6a5fb283c206e7d01effea87f68c8e   
800      9afec9408d028b77ab396251cbaac609  446430ddcdcf9a0cd80706a25ac7a45b   
819      96d46f4a620441197739c4d334364311  446430ddcdcf9a0cd80706a25ac7a45b   
1164     de4997efdcdfa69819e1b0794815bb41  c46542bcd3fb900fa42d57d03cafb727   
...                                   ...                               ...   
7790636  672be194ab66116720214a875208541f  72a4e269893e6471c9d35f4ee0587251   
7791125  4bee2cd0eb5568b44929635e27ed98ea  0c8b7c2a8bca48ba289cc589aad748e0   
7791212  4019534b6bad8cdada154605bae3738f  009126a6f164ab270cb6c106b6a58028   
7791339  c37822c30576714979c146f01c7c5863  e82685998023f680ddb24cad402164f7   
7791754  4d152f383e239d57783e88f742d0688f  40d24c92a11a6ba7eae7583641869b07   

         is_clk  is_like  is_addcart  is_order    e

In [40]:
train_agg_feat[['user_id', 'goods_id']].to_csv('u2i.csv', index=None)